In [2]:
import pandas as pd
from mplsoccer import Sbopen, Pitch
import numpy as np
import math

In [3]:
parser = Sbopen()
df_match = parser.match(competition_id=37, season_id=42)
match_ids = df_match.match_id.unique()

In [4]:
all_events_df = []
for match_id in match_ids:
    events = parser.event(match_id)[0]
    # Filter for shot events that are free kicks
    free_kick_shots = events[(events['type_name'] == 'Shot') & (events['sub_type_name'] == 'Free Kick')]
    all_events_df.append(free_kick_shots)
df_shot = pd.concat(all_events_df, ignore_index=True)
df_shot

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,...,block_deflection,shot_deflected,bad_behaviour_card_id,bad_behaviour_card_name,block_offensive,shot_follows_dribble,block_save_block,shot_redirect,half_start_late_video_start,half_end_early_video_end
0,e8f30e40-e373-4331-8407-6338db0781e1,1518,2,00:06:34.195000,51,34,119,1.640702,2275127,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a0466347-7d68-4ffd-85d8-5f094f6b8564,2618,2,00:22:34.292000,67,34,134,1.242266,2275136,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37f8ac32-73ca-4155-9d4a-beb3ae41fe58,1254,1,00:43:15.944000,43,15,90,1.457945,2275150,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50eddae9-9fc5-4478-a4b0-d2ebf48005e3,1942,2,00:25:41.100000,70,41,144,1.392752,2275150,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6bb2984d-cc19-4ab8-81ca-ef0e10dfe2c2,2197,2,00:37:35.289000,82,35,162,2.203068,2275150,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,f2ca6769-59f3-4226-83ec-09f268ac290a,13,1,00:00:39.474000,0,39,3,1.438998,2275120,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,7c2377a3-a0e9-4db2-a626-91b7fbacbbb4,540,1,00:12:05.188000,12,5,30,0.554275,2275028,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,3bd05aa0-23b3-4673-a6da-dcbadfb98d95,1509,1,00:42:38.170000,42,38,93,1.722035,2275086,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,a4558e68-fccf-4666-a49f-c4db2d344e3f,2727,2,00:34:49.338000,79,49,184,1.194096,2275086,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_shot.columns.to_list()
df_goals = df_shot[df_shot.outcome_name == 'Goal'][['x', 'y', 'outcome_name', 'shot_statsbomb_xg']].copy()


df_non_goal_shots = df_shot[df_shot.outcome_name != 'Goal'][['x', 'y', 'outcome_name', 'shot_statsbomb_xg']].copy()
df_non_goal_shots['outcome_name'] = 'No Goal'

In [34]:
def calculate_angle(x, y):
  # 44 and 36 is the location of each goal post
  g0 = [120, 44]
  p = [x, y]
  g1 = [120, 36]

  v0 = np.array(g0) - np.array(p)
  v1 = np.array(g1) - np.array(p)

  angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
  return(abs(np.degrees(angle)))

def calculate_distance(x, y):
  x_dist = 120-x
  y_dist = 0
  if (y<36):
    y_dist = 36-y
  elif (y>44):
    y_dist = y-44
  return math.sqrt(x_dist**2 + y_dist**2)

In [35]:
df_goals['distance'] = df_goals.apply(lambda row: calculate_distance(row['x'], row['y']), axis=1)
df_goals['angle'] = df_goals.apply(lambda row: calculate_angle(row['x'], row['y']), axis=1)

# Add distance and angle columns to df_non_goal_shots
df_non_goal_shots['distance'] = df_non_goal_shots.apply(lambda row: calculate_distance(row['x'], row['y']), axis=1)
df_non_goal_shots['angle'] = df_non_goal_shots.apply(lambda row: calculate_angle(row['x'], row['y']), axis=1)

C:\Users\rrayn\AppData\Local\Temp\ipykernel_16708\2619458180.py:10: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
C:\Users\rrayn\AppData\Local\Temp\ipykernel_16708\2619458180.py:10: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))


In [38]:
df = pd.concat([df_goals, df_non_goal_shots], axis=0)

# Reset index (optional)
df = df.reset_index(drop=True)
df


,x,y,outcome_name,shot_statsbomb_xg,distance,angle
0,93.6,75.6,Goal,0.005921,41.176692,6.186767
1,101.2,40.0,Goal,0.121448,18.800000,24.022957
2,97.6,51.2,Goal,0.074687,23.528706,16.340806
3,90.2,53.7,Goal,0.015563,31.338953,12.678364
4,100.5,26.2,Goal,0.068447,21.824069,15.708010
...,...,...,...,...,...,...
65,83.7,38.0,No Goal,0.012272,36.300000,12.539122
66,94.1,26.1,No Goal,0.041974,27.727604,13.730289
67,86.0,35.3,No Goal,0.020629,34.007205,13.173558
68,108.5,15.3,No Goal,0.008374,23.679949,7.218783


In [39]:
summary_stats = df.groupby('outcome_name')[['distance', 'angle']].mean().round(2)
print(summary_stats)

              distance  angle
outcome_name                 
Goal             27.17  15.24
No Goal          28.61  13.54
